### HELLENIC OPEN UNIVERSITY - SCHOOL OF SCIENCE AND TECHNOLOGY
### DATA SCIENCE AND MACHINE LEARNING : DAMA61 ACAD. YEAR 2023-24

#### <center> WRITTEN ASSIGNMENT 1 - SOLUTIONS </center>

In [ ]:
# increase the width of the notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

### Problem 1

Work with all the data of the data frame of Life satisfaction and the GDP per capita (full_country_stats).

1) Apply polynomial regression with a polynomial degree 8 and plot your results.</br>
2) Calculate the life satisfaction index (LSI) for a country with GDP equal to 97000.</br>
3) Use the three nearest neighbours to the country with GDP equal to 97000 to estimate its LSI.</br>
4) Compare the predictions of the two approaches. Comment on your results. </br>
5) Now, make predictions by using polynomials with degree from 1 to 10. What do you observe? Comment on your results.

In [ ]:
# import the needed packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline

# set the plotting parameters
%matplotlib inline
plt.rcParams.update({'font.size': 12})

In [ ]:
# download the data
import os
import urllib.request

download_root = "https://github.com/ageron/data/raw/main/"

datapath = os.path.join("datasets", "lifesat")
os.makedirs(datapath, exist_ok=True)

for filename in ("oecd_bli.csv", "gdp_per_capita.csv"):
    print(f"Downloading file: {filename}")
    url = download_root + "lifesat/" + filename
    urllib.request.urlretrieve(url, os.path.join(datapath, filename))

In [ ]:
# load the data
oecd_bli = pd.read_csv(os.path.join(datapath, "oecd_bli.csv"))
gdp_per_capita = pd.read_csv(os.path.join(datapath, "gdp_per_capita.csv"))

In [ ]:
# print the first 5 lines of the data in each file
oecd_bli.head()

In [ ]:
gdp_per_capita.head()

In [ ]:
# prepare the data
gdp_year = 2020
gdppc_col = "GDP per capita (USD)"
lifesat_col = "Life satisfaction"

gdp_per_capita = gdp_per_capita[gdp_per_capita["Year"] == gdp_year]
gdp_per_capita = gdp_per_capita.drop(["Code", "Year"], axis=1)
gdp_per_capita.columns = ["Country", gdppc_col]
gdp_per_capita.set_index("Country", inplace=True)

gdp_per_capita.head()

In [ ]:
oecd_bli = oecd_bli[oecd_bli["INEQUALITY"] == "TOT"]
oecd_bli = oecd_bli.pivot(index="Country", columns="Indicator", values="Value")

oecd_bli.head()

In [ ]:
full_country_stats = pd.merge(left=oecd_bli, right=gdp_per_capita,
                              left_index=True, right_index=True)
full_country_stats.sort_values(by=gdppc_col, inplace=True)
full_country_stats = full_country_stats[[gdppc_col, lifesat_col]]

full_country_stats.head()

In [ ]:
# create the traing set
Xfull = np.c_[full_country_stats["GDP per capita (USD)"]]
yfull = np.c_[full_country_stats["Life satisfaction"]]

In [ ]:
# Answer 1
poly = PolynomialFeatures(degree=8, include_bias=False) 
scaler = StandardScaler()
model2 = LinearRegression()

pl_reg = Pipeline([('poly', poly), ('scal', scaler), ('lin', model2)])
pl_reg.fit(Xfull, yfull)

full_country_stats.plot(kind="scatter", x="GDP per capita (USD)", 
                        y="Life satisfaction", figsize=(8,3))

plt.axis([0, 120000, -2, 10])

X = np.linspace(0, 120000, 1000)
plt.plot(X, pl_reg.predict(X.reshape(-1,1)), "k--")

# Answer 2
X0 = 97000
ls_X0 = pl_reg.predict([[X0]])
plt.plot(X0, ls_X0, "r*", markersize = 15)
plt.annotate(f"({X0}, {ls_X0[0][0]:.1f})", xy=(X0, ls_X0-0.25), xytext=(X0-28000, ls_X0-1.5),
             arrowprops=dict(facecolor="black", width=0.25, shrink=0.1, headwidth=6))

plt.grid()
plt.show()

In [ ]:
# Answer 3
X0 = 97000

# find the three nearest neighbours
X = abs(Xfull - X0)
indices = X.flatten().argsort()

# print the GDP and the life satisfaction index (LSI) of the three nearest neighbours
print("GDP:", Xfull[indices[:3]].flatten())
print("LSI:", yfull[indices[:3]].flatten())

# print the 
print(f"Predicted LSI: {np.mean(yfull[indices[:3]]):.1f}")

In [ ]:
# Answer 4

#### Comment:
By using a polynomial of degree 8 we estimate the LSI of a country with GDP equal to 97000 to be about 2. While by using the LSI of the three nearest neighbours (in terms of GDP) we predict a value of 7.1, which is a reasonable value. This result reveals that the polynomial of degree 8 fits the training data very closely (overfits) but fails to generalize well to areas where there are unknown data points.

In [ ]:
# Answer 5
full_country_stats.plot(kind="scatter", x="GDP per capita (USD)",
                        y="Life satisfaction", figsize=(8,6))

plt.axis([0, 120000, -2, 10])
X = np.linspace(0, 120000, 1000)

for poly_degree in range(1,11):
    poly = PolynomialFeatures(degree=poly_degree, include_bias=False) 
    scaler = StandardScaler()
    model2 = LinearRegression()

    pl_reg = Pipeline([('poly', poly), ('scal', scaler), ('lin', model2)])
    pl_reg.fit(Xfull, yfull)

    plt.plot(X, pl_reg.predict(X.reshape(-1,1)), color = f"C{poly_degree}", 
             ls="--", label=f"degree:{poly_degree}")

    X0 = 97000
    plt.plot(X0, pl_reg.predict([[X0]]), "r*", markersize = 15)

plt.legend()
plt.grid()
plt.show()

#### Comment:
By trying several values for the degree of a polynomial we observe that as the degree increases the model overfits the training data and again losses its ability to generalize well to unknown values of the GDP. This means that the model becomes less reliable in predicting life satisfaction for countries with GDP values that were not included in the training data. Therefore, it is important to find an appropriate balance in choosing the degree of the polynomial to prevent overfitting and ensure better generalization to unseen data.

<hr>

### Problem 2

Work with the data used to predict median house values in Californian districts (file: housing.tgz).

1) Use a Random Forest Regressor as your estimator and perform Grid Search cross-validation with parameters: </br>
n_estimators: [5, 15, 25], and max_features: [2, 4, 8].</br>
   What are the parameters of your best model? Are you satisfied with your results? </br>

2) What are the three most important features of your best model?

In [ ]:
# import the needed packages
from pathlib import Path
import tarfile
import urllib.request
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# load the data

def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))

housing = load_housing_data()

In [ ]:
# display the data
housing

In [ ]:
# create income_cat as a new feature based on the median_income
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

In [ ]:
# use stratified split to create the training and test sets
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
# create the target value of this problem and remove it from the input features 
housing_labels = strat_train_set['median_house_value'].copy()
housing = strat_train_set.drop(['median_house_value', 'income_cat'], axis=1) 

In [ ]:
# we need to dummify the categorical variable  
housing_cat = housing[["ocean_proximity"]].copy()
housing_num = housing.drop("ocean_proximity", axis=1)

In [ ]:
# prepare the numeric and the categorical features
num_features = list(housing_num)
cat_features = ["ocean_proximity"]

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")), # or "most_frequent"
        ('std_scaler', StandardScaler())
    ])

preprocessing = ColumnTransformer([
        ("num", num_pipeline, num_features),
        ("cat", OneHotEncoder(), cat_features)
    ])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

full_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    # create a Random Forest Regressor model as part of the pipeline
    ("random_forest", RandomForestRegressor(random_state=42))])

# set the parameters grid
param_grid = [
    {'random_forest__n_estimators': [5, 15, 25],
     'random_forest__max_features': [2, 4, 8]}]

# use GridSearchCV to apply 3-fold cv
grid_search = GridSearchCV(full_pipeline, param_grid, cv=3,
                           scoring="neg_root_mean_squared_error")

# fit the grid to find the best parameters
grid_search.fit(housing, housing_labels)

In [ ]:
# get the best parameters and the best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

In [ ]:
# print the best parameters
print (best_params)

#### Comments:
We observe that both the maximum number of features and the number of estimators of the best model take values equal to the upper limit of the values considered during the grid search. This result suggests that it may be beneficial to increase the upper limits of the hyperparameters and rerun the calculation in order to potentially achieve better results. Therefore, we can conclude that the current results are unsatisfactory.

In [ ]:
feature_importances = best_model['random_forest'].feature_importances_

sorted(zip(feature_importances,
           best_model['preprocessing'].get_feature_names_out()),
           reverse=True)

#### Comments:
Based on our analysis and the best parameters, we conclude that the three most important features are median income, INLAND, and longitude.

<hr>

### Problem 3

Work with the MNIST dataset and:

1) Load the data and split them into training (6/7) and test (1/7) sets.</br>
2) Train a binary classifier of your choice to distinguish between two classes, 4 and not-4.</br>
3) Use 3-fold cross validation and evaluate your model by calculating the metrics: accuracy, recall, and precision. Compare the accuracy of your model to the accuracy of a model that always guesses that an image is not a 4.</br>
4) Calculate the confusion matrix for the train set. How many of the train samples were wrongly classified as 4s and how many wrongly classified as non-4s?</br>
5) Plot the ROC curve and calculate the area under the curve (AUC).

In [ ]:
# load the MNIST dataset
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame=False)

In [ ]:
# split the data into features and target values
X, y = mnist["data"], mnist["target"]

In [ ]:
# split the dataset to training (6/7) and test sets (1/7)
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [ ]:
# display the first digit and its label value of the training set
plt.imshow(X_train[0].reshape((28,28)))
plt.show()
print(f"This is a {y_train[0]}")

In [ ]:
# create the 4 and not-4 target values
y_train_4 = (y_train == '4')
y_test_4 = (y_test == '4')

In [ ]:
# display the first digit and its label value of the training set
plt.imshow(X_train[0].reshape((28,28)))
plt.show()
print(f"This is {'a four' if y_train_4[0] else 'not a four'}")

In [ ]:
# train an SGD classifier 
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)
sgd_clf.fit(X_train, y_train_4)

In [ ]:
# calculate the accuracy, recall and precision of the training set
from sklearn.model_selection import cross_validate
scores = cross_validate(sgd_clf, X_train, y_train_4, cv=3,
                        scoring=['accuracy', 'recall', 'precision'])

In [ ]:
print(f"Accuracy of each fold: {scores['test_accuracy']}, mean accuracy: {100*scores['test_accuracy'].mean():.1f}%")
print(f"Recall of each fold: {scores['test_recall']}, mean recall: {100*scores['test_recall'].mean():.1f}%")
print(f"Precision of each fold: {scores['test_precision']}, mean precision: {100*scores['test_precision'].mean():.1f}%")

In [ ]:
# an easy way to calculate the accuracy of a model that always guesses an image is not a 4, you can sum up all the correct guesses,
# i.e., the non-4 images in the dataset, and divide that by the total number of images. 
non_4_accuracy = (y_train_4 == 0).sum()/len(y_train_4)
print (f"Accuracy of a model that always guesses not a 4: {100*non_4_accuracy:.1f}%")

#### Comment:
The accuracy of the model that always guesses an image is not a 4 is 90.3%.</br>
This example demonstrates that when evaluating models, it's crucial to consider the balance of the dataset and be cautious when using accuracy as the sole metric.</br>
Accuracy can be misleading if the classes are imbalanced, meaning one class is significantly more prevalent than the other. In such cases, a model that always predicts the majority class can achieve high accuracy even without truly learning the underlying patterns.</br>
To overcome this limitation, it's important to consider additional evaluation metrics, such as precision, recall, F1 score, or area under the receiver operating characteristic curve (AUC-ROC). These metrics provide a more comprehensive assessment of model performance, especially when dealing with imbalanced datasets.

In [ ]:
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_4, cv=3)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_4, y_train_pred)

#### Comment:
The confusion matrix shows that the model made 52,957 correct negative (TN) predictions, 1,201 false positive (FP) predictions, 554 false negative (FN) predictions, and 5,288 correct positive (TP) predictions.

In [ ]:
y_scores = cross_val_predict(sgd_clf, X_train, y_train_4, cv=3, method="decision_function")

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_train_4, y_scores)

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.figure(figsize=(8, 6))                  
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--', label="Random")
    plt.axis([0, 1, 0, 1])              
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend()
    plt.grid()

plot_roc_curve(fpr, tpr, "Model")

In [ ]:
# calculate the area under the curve
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train_4, y_scores)

#### Comment:
Both the ROC and the area under the curve demonstrate that our model performs well on the training data.